<a href="https://colab.research.google.com/github/ChrisGold/DM873-Project-2/blob/main/Lungs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
! ls /content/drive/My\ Drive/data

allData  eval  train  val


In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Conv2D, MaxPool2D, Input, Dropout, experimental, AveragePooling2D, GlobalAveragePooling2D

#we use a quite large patience for early stopping, because we have a lot of fluctuation.

patience = 10
imageSizeX = 224
imageSizeY = 224
batchSize = 32

TrainDataGen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,fill_mode = 'nearest',
    
    channel_shift_range=13,data_format='channels_last',
)

ValDataGen=ImageDataGenerator(rescale=1./255)


TrainData = ValDataGen.flow_from_directory(batch_size = batchSize,
                                                 directory = 'drive/My Drive/lung/train/',
                                                 shuffle = True,
                                                 target_size = (imageSizeX, imageSizeY), 
                                                 subset = "training",
                                                 color_mode="grayscale",
                                                 class_mode = 'binary')

ValData = ValDataGen.flow_from_directory(batch_size = batchSize,
                                                 directory = 'drive/My Drive/lung/val/',
                                                 shuffle = True,
                                                 target_size = (imageSizeX, imageSizeY), 
                                                 subset = "training",
                                                 color_mode="grayscale",
                                                 class_mode = 'binary')




# stops the training at given training accuracy
class myCallback(keras.callbacks.Callback): 
    targetAccuracy = 0.9
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('accuracy') > (self.targetAccuracy)):   
            self.model.stop_training = True

            


cb = keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=patience, verbose=0, mode='auto',
    baseline=None, restore_best_weights=True
)


model = keras.Sequential([

    
    layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(imageSizeX, imageSizeY, 1)),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.BatchNormalization(),
    layers.Dropout(0.4),


    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.BatchNormalization(),
    layers.Dropout(0.4),

    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.BatchNormalization(),
    layers.Dropout(0.4),

    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.BatchNormalization(),
    layers.Dropout(0.4),

    layers.Conv2D(256, 3, padding='same', activation='relu'),
    layers.Conv2D(256, 3, padding='same', activation='relu'),
    layers.Conv2D(256, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.BatchNormalization(),
    layers.Dropout(0.4),

    layers.Conv2D(256, 3, padding='same', activation='relu'),
    layers.Conv2D(256, 3, padding='same', activation='relu'),
    layers.Conv2D(256, 3, padding='same', activation='relu'),
    layers.GlobalMaxPooling2D(),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),

    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),    

    layers.Dense(1, activation ='sigmoid')
])


model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history =model.fit(TrainData, 
                epochs=1000, #we stop training the model with early stopping
                validation_data=ValData,
                callbacks=[cb])


# after the training on the training dataset is done, we attempt to learn on the validation data + training data
# until we reach the same training accuracy as we did on the training data

#allData = TrainDataGen.flow_from_directory(batch_size=batchSize,
#                                                 directory='drive/My Drive/data/allData/',
#                                                 shuffle=True,
#                                                 target_size=(imageSizeX, imageSizeY), 
#                                                 subset="training",
#                                                 class_mode='binary')


#cb = myCallback()
#cb.targetAccuracy = history.history['accuracy'][-(patience+1)]
#print(cb.targetAccuracy)
#
#model.fit(allData, 
#                    epochs=1000, 
#                    callbacks=[cb])

model.save('lungs_auto.h5')



Found 2382 images belonging to 2 classes.
Found 300 images belonging to 2 classes.
Epoch 1/1000
75/75 [==============================] - 40s 537ms/step - loss: 0.4503 - accuracy: 0.8363 - val_loss: 1.6228 - val_accuracy: 0.5000
Epoch 2/1000
75/75 [==============================] - 39s 524ms/step - loss: 0.2459 - accuracy: 0.9148 - val_loss: 4.0636 - val_accuracy: 0.5000
Epoch 3/1000
75/75 [==============================] - 39s 525ms/step - loss: 0.1789 - accuracy: 0.9307 - val_loss: 5.8914 - val_accuracy: 0.5000
Epoch 4/1000
75/75 [==============================] - 39s 525ms/step - loss: 0.1552 - accuracy: 0.9467 - val_loss: 1.7857 - val_accuracy: 0.6500
Epoch 5/1000
75/75 [==============================] - 40s 530ms/step - loss: 0.1361 - accuracy: 0.9521 - val_loss: 0.7007 - val_accuracy: 0.8633
Epoch 6/1000
75/75 [==============================] - 40s 528ms/step - loss: 0.1065 - accuracy: 0.9656 - val_loss: 0.5145 - val_accuracy: 0.8967
Epoch 7/1000
75/75 [===========================

In [12]:
testDataGen=ImageDataGenerator(rescale=1./255)

test = testDataGen.flow_from_directory(batch_size = batchSize,
                                                 directory = 'drive/My Drive/lung/val/',
                                                 shuffle = False,
                                                 target_size = (imageSizeX, imageSizeY), 
                                                 subset = "training",
                                                 color_mode="grayscale",
                                                 class_mode = 'binary')

#modelLoad = keras.models.load_model('cats_dogs_auto2.h5')
model.evaluate(test)


Found 300 images belonging to 2 classes.
20/20 [==============================] - 6s 303ms/step - loss: 1.5666 - accuracy: 0.8176


[1.5666486024856567, 0.8175895810127258]